# Homework 8-1

---

## Imports

In [1]:
from thermostate import State, Q_, units, SystemInternational as SI

---

## Definitions

In [2]:
sub_v = 'water'
sub_a = 'air'

T_1 = Q_(15.0, 'degC')
p_1 = Q_(1.3, 'atm')
phi_1 = Q_(0.63, 'dimensionless')
Vdot_1 = Q_(773.0, 'm**3/hr')

T_w = Q_(15.0, 'degC')
mdot_w = Q_(7.0, 'kg/hr')

T_3 = Q_(45.0, 'degC')
p_3 = Q_(1.3, 'atm')

---

## Problem Statement

Moist air at 15.0 celsius, 1.3 atm, 63.00%  relative humidity and a volumetric flow rate of 773.0 m<sup>3</sup>/hr enters a control volume and flows along a high-temperature surface such that energy is transferred by heat to the moist air. A separate stream of liquid water at 15.0 celsius is injected at a rate of 7.0 kg/hr and evaporates completely in the air stream. The stream exits the control volume at 45.0 celsius, 1.3 atm. Determine the rate of heat transfer into the flowing stream, in kW.

---

## Solution

To find the rate of heat transfer input to the flowing stream, we need to write the energy balance for a control volume that contains the two inlets and one outlet. The mass rate balances for the components show that the mass flow rate of air is equal at the inlet and outlet, while the mass flow rate of water at the outlet is the sum of the mass flow rate of the vapor at the inlet and the mass flow rate of the liquid water. Therefore, the energy balance works out to
$$0 = \dot{Q}_{cv} + \dot{m}_a h_{a1} + \dot{m}_{v1} h_{v1} + \dot{m}_{w} h_{w} - \dot{m}_a h_{a3} - \dot{m}_{v3} h_{v3}$$

To solve this equation for the rate of heat transfer, we need to find the enthalpies of air at the inlet and outlet, the enthalpies of the water vapor at the inlet and outlet, and the enthalpy of the liquid water. For the mass flow rates, we can re-arrange in the following way
$$\dot{m}_{w} = \dot{m}_{v3} - \dot{m}_{v1}$$

Then, dividing through by $\dot{m}_a$,
$$\frac{\dot{m}_{w}}{\dot{m}_a} = \omega_3 - \omega_1$$
where
$$\frac{\dot{m}_{v3}}{\dot{m}_a} = \omega_3$$
and
$$\frac{\dot{m}_{v1}}{\dot{m}_a} = \omega_1$$

Plugging back into the energy equation, we find that
$$\dot{Q}_{cv} = \dot{m}_a \left[h_{a3} - h_{a1} + \omega_3 h_{v3} - \omega_1 h_{v1} + (\omega_1 - \omega_3)h_{w}\right]$$

First, we will solve for the value of $\omega_1$, then $\dot{m}_a$ and $\omega_3$, then we will fix all the states to get the enthalpies, and then we can find the rate of heat transfer.

To find $\omega_1$, we need the partial pressure of the water vapor at the inlet. This can be found from the given temperature and relative humidity,
$$p_{v1} = \phi_1 p_{g1}$$

In [3]:
st_g1 = State(sub_v, T=T_1, x=Q_(1.0, 'dimensionless'))
p_g1 = st_g1.p.to(SI.p)

p_v1 = phi_1*p_g1

The saturation pressure at the inlet is $p_{g1} =$ 0.0171 bar, while the partial pressure at the inlet is $p_{v1} =$ 0.0107 bar. To find $\omega_3$, we need the mass flow rate of air, which can be found by dividing the volumetric flow rate by the specific volume of the air, found by the ideal gas law.

In [4]:
MW_a = Q_(28.97, 'kg/kmol')
p_a1 = p_1 - p_v1
v_a1 = ((units.molar_gas_constant/MW_a)*T_1/p_a1).to('m**3/kg')
mdot_a = Vdot_1/v_a1

The partial pressure of the air is $p_{a1} =$ 1.29 atm, the specific volume is $v_{a1} =$ 0.63 m<sup>3</sup>/kg, and the mass flow rate of air is $\dot{m}_a =$ 1221.17 kg/hr. Next, the humidity ratio of the mixture at the outlet can be found by rearranging
$$\omega_3 = \frac{\dot{m}_{w}}{\dot{m}_a} + \omega_1$$
and
$$\omega_1 = 0.622 \frac{p_{v1}}{p_1 - p_{v1}}$$

In [5]:
MW_v = Q_(18.02, 'kg/kmol')
omega_1 = MW_v/MW_a*p_v1/(p_1 - p_v1)
omega_3 = omega_1 + (mdot_w/mdot_a).to('dimensionless')

The value for $\omega_1 =$ 0.00518  and $\omega_3 =$ 0.011 . We can double check that all the liquid water has evaporated by finding the relative humidity at the outlet and ensuring it is less than 100%. We need the saturation pressure at $T_3$, and $p_{v3}$ from $\omega_3$.
$$p_{v3} = \frac{\omega_3 p_3}{0.622 + \omega_3}$$

In [6]:
p_v3 = (omega_3*p_3/(MW_v/MW_a + omega_3)).to(SI.p)

st_g3 = State(sub_v, T=T_3, x=Q_(1.0, 'dimensionless'))
p_g3 = st_g3.p.to(SI.p)

phi_3 = p_v3/p_g3

The relative humidity at the outlet is $\phi_3 =$ 0.24  = 23.53% , validating our assumption that everything is vaporized at the outlet. Now the enthalpies are evaluated. We fix the states at each condition, assuming that the values for liquid water can be given by the saturated liquid values.

In [7]:
st_a1 = State(sub_a, T=T_1, p=p_a1)
h_a1 = st_a1.h.to(SI.h)

st_v1 = State(sub_v, T=T_1, p=p_v1)
h_v1 = st_v1.h.to(SI.h)

st_a3 = State(sub_a, T=T_3, p=(p_3 - p_v3))
h_a3 = st_a3.h.to(SI.h)

st_v3 = State(sub_v, T=T_3, p=p_v3)
h_v3 = st_v3.h.to(SI.h)

st_w = State(sub_v, T=T_w, x=Q_(0.0, 'dimensionless'))
h_w = st_w.h.to(SI.h)

Summarizing the states,

| State |           $h_a$            |           $h_v$            |           $h_w$           |
|-------|----------------------------|----------------------------|---------------------------|
| 1     | 414.30 kJ/kg | 2528.75 kJ/kg | ---                       |
| w     | ---                        | ---                        | 62.98 kJ/kg |
| 3     | 444.51 kJ/kg | 2584.67 kJ/kg | ---                       |

Then, the heat transfer rate can be calculated

In [8]:
Qdot_cv = (mdot_a*(h_a3 - h_a1 + omega_3*h_v3 - omega_1*h_v1 + (omega_1 - omega_3)*h_w)).to('kW')

<div class="alert alert-success">

**Answer:** The rate of heat transfer is $\dot{Q}_{cv} =$ 15.25 kW

</div>